In [2]:
import os
import zipfile
# name of the zip file you want to unzip
local_zip = 'glaucoma-detection.zip'
# opening a file with mode parameter 'r' : read existing file
zip_ref = zipfile.ZipFile(local_zip, 'r')
# extract all contents of the zip file
zip_ref.extractall('')
# close the file
zip_ref.close()

In [8]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('glaucoma.csv')
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,OD,A,0
1,002.jpg,0.6953,OS,A,0
2,003.jpg,0.9629,OS,A,0
3,004.jpg,0.7246,OD,A,0
4,005.jpg,0.6138,OS,A,0


In [3]:
df.describe(include='all')


,Filename,ExpCDR,Eye,Set,Glaucoma
count,650,650.000000,650,650,650.000000
unique,650,NaN,2,2,NaN
top,090.jpg,NaN,OD,A,NaN
freq,1,NaN,327,325,NaN
mean,NaN,0.576308,NaN,NaN,0.258462
std,NaN,0.116395,NaN,NaN,0.438126
min,NaN,0.161000,NaN,NaN,0.000000
25%,NaN,0.496675,NaN,NaN,0.000000
50%,NaN,0.567650,NaN,NaN,0.000000
75%,NaN,0.648475,NaN,NaN,1.000000


In [4]:
df['Eye'].unique()


array(['OD', 'OS'], dtype=object)

In [5]:
df['eye'] = df['Eye'].map({'OD': 0, 'OS': 1})
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye
0,001.jpg,0.7097,OD,A,0,0
1,002.jpg,0.6953,OS,A,0,1
2,003.jpg,0.9629,OS,A,0,1
3,004.jpg,0.7246,OD,A,0,0
4,005.jpg,0.6138,OS,A,0,1


In [6]:
df['set'] = df['Set'].map({'A': 0, 'B': 1})
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye,set
0,001.jpg,0.7097,OD,A,0,0,0
1,002.jpg,0.6953,OS,A,0,1,0
2,003.jpg,0.9629,OS,A,0,1,0
3,004.jpg,0.7246,OD,A,0,0,0
4,005.jpg,0.6138,OS,A,0,1,0


In [9]:
df['image'] = np.nan
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye,set,image
0,001.jpg,0.7097,OD,A,0,0,0,NaN
1,002.jpg,0.6953,OS,A,0,1,0,NaN
2,003.jpg,0.9629,OS,A,0,1,0,NaN
3,004.jpg,0.7246,OD,A,0,0,0,NaN
4,005.jpg,0.6138,OS,A,0,1,0,NaN


In [15]:
image_dir = 'Fundus_Train_Val_Data/Fundus_Scanes_Sorted'


In [16]:
train_dir = os.path.join(image_dir, 'Train')
validation_dir = os.path.join(image_dir, 'Validation')

In [17]:
mapping = {
    0: 'Glaucoma_Negative',
    1: 'Glaucoma_Positive'
}


In [18]:
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
train_image_negative = os.listdir(os.path.join(train_dir, 'Glaucoma_Negative'))
train_image_positive = os.listdir(os.path.join(train_dir, 'Glaucoma_Positive'))
print(len(train_image_negative) + len(train_image_positive))


520


In [20]:
test_image_negative = os.listdir(os.path.join(validation_dir, 'Glaucoma_Negative'))
test_image_positive = os.listdir(os.path.join(validation_dir, 'Glaucoma_Positive'))
print(len(test_image_negative) + len(test_image_positive))

130


In [21]:
df_train = df[df['Filename'].isin(train_image_negative) | df['Filename'].isin(train_image_positive)]
len(df_train)

520

In [22]:
df_test = df[df['Filename'].isin(test_image_negative) | df['Filename'].isin(test_image_positive)]
len(df_test)

130

In [23]:
df_train['Glaucoma'].value_counts()

0    386
1    134
Name: Glaucoma, dtype: int64

In [24]:
IMAGE_SIZE = (224,224)


In [25]:
def load_image(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(train_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [27]:
df_train['image'] = df_train.apply(load_image, axis=1)


KeyboardInterrupt: 

In [ ]:
def load_image1(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(validation_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [ ]:
df_test['image'] = df_test.apply(load_image1, axis=1)


In [ ]:
X_image_train = df_train['image']
X_data_train = df_train[['ExpCDR', 'eye', 'set']]
y_train = df_train['Glaucoma']

In [ ]:
X_image_test = df_test['image']
X_data_test = df_test[['ExpCDR', 'eye', 'set']]
y_test = df_test['Glaucoma']

In [ ]:
X_image_train_stacked = np.stack(X_image_train.values)
X_image_test_stacked = np.stack(X_image_test.values)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights

In [ ]:
class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1]
}
class_weight_dict